# fashion MNIST  
---  
## Training  
https://www.youtube.com/watch?v=XfYmia3q2Ow&list=PLZbbT5o_s2xrfNyHZsM6ufI0iZENK9xgG&index=26

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
# from torchvision import transforms

torch.set_printoptions(linewidth=120) # display option for output
torch.set_grad_enabled(True)

In [2]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        # linear layer == fully connected layer == fc == dense layer
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer:
        # t = t
        
        # (2) hidden conv layer:
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) hidden conv layer:
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer:
        t = t.reshape(-1, 12*4*4)
        # t = t.flatten()
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer:
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer:
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t

In [4]:
train_set = torchvision.datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [5]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(5):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels) # calculating the loss function

        optimizer.zero_grad() # zero out the gradients, because pytorch is actually adding the grads
        loss.backward() # calculating the gradients
        optimizer.step() # update the weight

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(f'epoch: {epoch} total correct: {total_correct} loss: {total_loss:.2f} accuracy {100*total_correct / len(train_set):.2f}%')


epoch: 0 total correct: 46726 loss: 348.69 accuracy 77.88%
epoch: 1 total correct: 51022 loss: 241.70 accuracy 85.04%
epoch: 2 total correct: 51860 loss: 218.15 accuracy 86.43%
epoch: 3 total correct: 52166 loss: 209.32 accuracy 86.94%
epoch: 4 total correct: 52313 loss: 205.73 accuracy 87.19%
